In [ ]:
# Include relevant packages
# Reduce the minimum methods in the notebook
# Put everything in pyACC

import sys
sys.path.append('../')

from pyACC.cosmology.distances import hubble_function, CosmologicalDistances, LambdaCDMCosmology

from pyACC.cosmology.distances_func import distance_modulus_from_redshift

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import pandas as pd

In [ ]:
# Theory

# Define your functions to get cosmological distances from a list of redshifts and different
# cosmological models
# 1. LCDM
# 2. wCDM
# 3. your choice

In [ ]:
print('LambdaCDM cosmo is ', LambdaCDMCosmology())

In [ ]:
cd = CosmologicalDistances(hubble_function)

In [ ]:
redshift_list = np.linspace(0.01, 2.2, 100)

mu = cd.distance_modulus_from_redshift(redshift_list)
luminosity_distance = cd.luminosity_distance(redshift_list)
hubble_distance = cd.hubble_distance(redshift_list)
transverse_comoving_distance = cd.transverse_comoving_distance(redshift_list)
isotropic_volume_distance = cd.isotropic_volume_distance(redshift_list)

In [ ]:
# Dataset

# Read the datasets, and plot the data against Planck 2018 LCDM cosmology

In [ ]:
print("Read SN data")
SN_data = pd.read_csv("./data/SNIa_data.txt", sep=" ", header=None, names=["Redshift", "Distance"])
SN_covariance = np.genfromtxt("./data/SNIa_covariance.txt")
SN_data["Error"] = np.sqrt(np.diag(SN_covariance))
SN_data["Kind"] = "\mu"
SN_data["Name"] = "Pantheon+"


In [ ]:
SN_data

In [ ]:
plt.plot(SN_data['Redshift'], SN_data['Distance'], label='SN_data')
plt.plot(redshift_list, mu, label=r'$\Lambda$CDM' )
plt.legend()

In [ ]:
print("Read BAO data")
BAO_data = pd.read_csv("./data/BAO_data.txt")
BAO_covariance = pd.read_csv("./data/BAO_covariance.txt", header=None).replace(np.NaN, 0).to_numpy()

# Sound horizon from Planck 2018, Table 2, TT,TE,EE+lowE
rd = 147.06

In [ ]:
BAO_data

In [ ]:
distance_type = pd.unique(BAO_data["Kind"])                 ## list of unique distance types

fig, ax = plt.subplots(1, 4, figsize=(21, 5))
ax[0].plot(redshift_list, isotropic_volume_distance/rd, color="g")
ax[1].plot(redshift_list, transverse_comoving_distance/rd, color="b")
ax[2].plot(redshift_list, hubble_distance/rd, color="r")
ax[3].plot(redshift_list, 5*np.log10(luminosity_distance), color="k")

## for each distance type plot, add errorbars and axis labels
for i,dt in enumerate(distance_type):
    data = BAO_data[BAO_data["Kind"] == dt]
    ax[i].errorbar(data["Redshift"], data["Distance"], yerr=data["Error"], fmt='o', color="k")
    ax[i].set_xlabel("Redshift")
    ax[i].set_ylabel(r"$%s$"%dt)

ax[3].errorbar(SN_data["Redshift"], SN_data["Distance"], yerr=SN_data["Error"], fmt='o', color="0.5", alpha=0.5)
ax[3].set_xlabel("Redshift")
ax[3].set_ylabel(r"$\mu$")

ax[0].set_title("BAO - Isotropic Volume Distance")
ax[1].set_title("BAO - Transverse Distance")
ax[2].set_title("BAO - Hubble Distance")
ax[3].set_title("SNIa - Distance Modulus")


In [ ]:
# Likelihood

# Define the likelihood function for the data

The likelihood for sn1a can be marginalized analitically on the intrinsic luminosity of the stars. See Marco 2022 'Cosmic Structures' class. 

$\mathcal{L}_m = \int \exp{[-\frac{1}{2}(d-\mu+\alpha\vec{1})^T\Sigma^{-1}(d-\mu+\alpha\vec{1})]d\alpha}$

The result is the following (after some matrix algebra and integrations, see matrix cookbook)

$\log{\mathcal{L}_m} = -\frac{1}{2}(d-\mu)^T\Sigma^{-1}(d-\mu)+\frac{1}{2}\frac
{[(d-\mu)^T\Sigma^{-1}\vec{1}]^2}{\vec{1}^T\Sigma^{-1}\vec{1}}$

In [31]:
mu_teo = distance_modulus_from_redshift(SN_data['Redshift'])
SN_inv_cov = np.linalg.inv(SN_covariance)
n = len(SN_data['Distance'])

def logL(pars):
    w=pars[0]
    Om_m=pars[1]
    x = SN_data['Distance']-distance_modulus_from_redshift(SN_data['Redshift'], Om_m=Om_m, w0=w)
    return -0.5*(-np.linalg.multi_dot([x,SN_inv_cov,x])+(np.linalg.multi_dot([x,SN_inv_cov,np.ones(n)]))**2/np.linalg.multi_dot([np.ones(n),SN_inv_cov,np.ones(n)]))

In [41]:
# Likelihood maximization
# Use scipy.optimize to maximize the likelihood function for different cosmological models
# Get best-fit parameters and plot the best-fit model against the data and Planck 2018 cosmology
in_guess = [-1,0.3]
max_l=minimize(logL,in_guess, bounds=[(-1.5,0.5),(0.1,1)])

In [43]:
print('maximum likelihood in', max_l.x)

maximum likelihood in [-1.02091276  0.29997543]


In [ ]:
# Model selection
# Test different models using the likelihood ratio test

In [ ]:
# Sample the posterior

# Define the priors for the cosmological parameters
# Sample the posterior using MCMC in pyACC
# Clean the chains and plot the results

In [ ]:
# Analyze the results

# Get relevant quantities from marginalized posteriors
# Check consistency with maximum likelihood results
# Check consistency with Planck 2018 results
# Plot the chains
# Plot the posterior distributions

In [ ]:
# Summarize your results

# Write a report with your results
# put emphasis on the numerical part, specyfing the numerical techniques used.
# Share your results in form of chains